In [1]:
# import the necessary libraries
import pandas as pd
import numpy as np
from sklearn import preprocessing
from numpy import set_printoptions
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import dtale

from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

C:\Users\user\AppData\Local\Temp\ipykernel_86452\697499791.py:19: DeprecationWarning:

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display



In [2]:

# read the data 
df = pd.read_excel('House_Rent_Dataset Excel.xlsx')
df.columns = df.columns.str.upper()   #convert the columns to uppercase 
df.head()

,POSTED ON,BHK,RENT,SIZE,FLOOR,AREA TYPE,AREA LOCALITY,CITY,FURNISHING STATUS,TENANT PREFERRED,BATHROOM,POINT OF CONTACT
0,2022-05-16,2,200000,1000,7 out of 8,Carpet Area,Union Park,Mumbai,Furnished,Bachelors/Family,2,Contact Agent
1,2022-05-14,3,200000,1208,5 out of 14,Carpet Area,Khar West,Mumbai,Unfurnished,Bachelors,3,Contact Agent
2,2022-07-09,3,200000,1350,1 out of 5,Carpet Area,anjali,Mumbai,Furnished,Bachelors,3,Contact Agent
3,2022-07-09,3,200000,1350,1 out of 5,Carpet Area,Anand niwas,Mumbai,Furnished,Bachelors,3,Contact Agent
4,2022-06-04,3,200000,1375,15 out of 60,Carpet Area,"Raheja Imperia, Worli",Mumbai,Semi-Furnished,Family,3,Contact Agent


In [3]:
dtale.show(df).open_browser()

C:\Users\user\anaconda3\Lib\site-packages\seaborn\matrix.py:309: UserWarning:

Attempting to set identical low and high xlims makes transformation singular; automatically expanding.

C:\Users\user\anaconda3\Lib\site-packages\seaborn\matrix.py:309: UserWarning:

Attempting to set identical low and high ylims makes transformation singular; automatically expanding.

2023-12-26 08:32:06,712 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-12-26 08:32:06,733 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-12-26 08:32:06,743 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-12-26 08:32:06,746 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-12-26 08:32:06,749 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-12-26 08:32:06,753 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-12-26 08:32:06,757 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-12-26 08:32:06,760 - WARNING  - findfont: Font family 'Heiti T

2023-12-26 08:32:11,166 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-12-26 08:32:11,170 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-12-26 08:37:17,932 - ERROR    - Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^

In [5]:

column_order = [col for col in df.columns if col != 'RENT'] + ['RENT']
df = df[column_order]
df.head()

,POSTED ON,BHK,SIZE,FLOOR,AREA TYPE,AREA LOCALITY,CITY,FURNISHING STATUS,TENANT PREFERRED,BATHROOM,POINT OF CONTACT,RENT
0,2022-05-16,2,1000,7 out of 8,Carpet Area,Union Park,Mumbai,Furnished,Bachelors/Family,2,Contact Agent,200000
1,2022-05-14,3,1208,5 out of 14,Carpet Area,Khar West,Mumbai,Unfurnished,Bachelors,3,Contact Agent,200000
2,2022-07-09,3,1350,1 out of 5,Carpet Area,anjali,Mumbai,Furnished,Bachelors,3,Contact Agent,200000
3,2022-07-09,3,1350,1 out of 5,Carpet Area,Anand niwas,Mumbai,Furnished,Bachelors,3,Contact Agent,200000
4,2022-06-04,3,1375,15 out of 60,Carpet Area,"Raheja Imperia, Worli",Mumbai,Semi-Furnished,Family,3,Contact Agent,200000


In [6]:
df[['FLOOR']] = df['FLOOR'].str.extract(r'(\d+) out of ')
df = df.fillna(value=0)
df['FLOOR'] = df['FLOOR'].astype(int)
df.head()

,POSTED ON,BHK,SIZE,FLOOR,AREA TYPE,AREA LOCALITY,CITY,FURNISHING STATUS,TENANT PREFERRED,BATHROOM,POINT OF CONTACT,RENT
0,2022-05-16,2,1000,7,Carpet Area,Union Park,Mumbai,Furnished,Bachelors/Family,2,Contact Agent,200000
1,2022-05-14,3,1208,5,Carpet Area,Khar West,Mumbai,Unfurnished,Bachelors,3,Contact Agent,200000
2,2022-07-09,3,1350,1,Carpet Area,anjali,Mumbai,Furnished,Bachelors,3,Contact Agent,200000
3,2022-07-09,3,1350,1,Carpet Area,Anand niwas,Mumbai,Furnished,Bachelors,3,Contact Agent,200000
4,2022-06-04,3,1375,15,Carpet Area,"Raheja Imperia, Worli",Mumbai,Semi-Furnished,Family,3,Contact Agent,200000


In [7]:

# df = df.loc[df['BATHROOM'] != 10]
# df = df.loc[df['POINT OF CONTACT'] != 'Contact Builder']
df = df.drop(['POSTED ON', 'AREA LOCALITY'], axis=1)
df.head(2)
dtale.show(df).open_browser()

In [11]:
def detectOutliers():
    fig, axs = plt.subplots(1,4, figsize = (6,4))
    plt1 = sns.boxplot(df['BHK'], ax = axs[0])
    plt1.set(title='BHK ')
    plt2 = sns.boxplot(df['SIZE'], ax = axs[1])
    plt2.set(title='Size')
    plt3 = sns.boxplot(df['BATHROOM'], ax = axs[2])
    plt3.set(title='Bathroom')
    plt4 = sns.boxplot(df['FLOOR'], ax = axs[3])
    plt4.set(title='Floor')

    plt.tight_layout()
detectOutliers()

KeyError: 0

In [8]:
# Outlier reduction for BHK
Q1 = df.BHK.quantile(0.25)
Q3 = df.BHK.quantile(0.75)
IQR = Q3 - Q1
df = df[(df.BHK >= Q1 - 1.5*IQR) & (df.BHK <= Q3 + 1.5*IQR)]

# Outlier reduction for Bathroom
Q1 = df.BATHROOM.quantile(0.25)
Q3 = df.BATHROOM.quantile(0.75)
IQR = Q3 - Q1
df = df[(df.BATHROOM >= Q1 - 1.5*IQR) & (df.BATHROOM <= Q3 + 1.5*IQR)]

# Outlier reduction for Floor
Q1 = df.FLOOR.quantile(0.25)
Q3 = df.FLOOR.quantile(0.75)
IQR = Q3 - Q1
df = df[(df.FLOOR >= Q1 - 1.5*IQR) & (df.FLOOR <= Q3 + 1.5*IQR)]

Next step visualizing the categorical variables

In [17]:

plt.figure(figsize=(20, 12))
plt.subplot(3,3,1)
sns.boxplot(x='FLOOR', y='RENT', data=df)
plt.subplot(3,3,2)
sns.boxplot(x='AREA TYPE', y='RENT', data=df)
plt.subplot(3,3,3)
sns.boxplot(x='POINT OF CONTACT', y='RENT', data=df)
plt.subplot(3,3,4)
sns.boxplot(x='CITY', y='RENT', data=df)
plt.subplot(3,3,5)
sns.boxplot(x='FURNISHING STATUS', y='RENT', data=df)
plt.subplot(3,3,6)
sns.boxplot(x='TENANT PREFERRED', y='RENT', data=df)
plt.show()

<Figure size 2000x1200 with 6 Axes>

In [18]:
cat_columns = ['FLOOR','AREA TYPE','CITY','FURNISHING STATUS','TENANT PREFERRED', 'POINT OF CONTACT']
df_encoded = pd.get_dummies(df, columns=cat_columns)

In [19]:
df_encoded.head()

,BHK,SIZE,BATHROOM,RENT,FLOOR_0,FLOOR_1,FLOOR_2,FLOOR_3,FLOOR_4,FLOOR_5,...,CITY_Mumbai,FURNISHING STATUS_Furnished,FURNISHING STATUS_Semi-Furnished,FURNISHING STATUS_Unfurnished,TENANT PREFERRED_Bachelors,TENANT PREFERRED_Bachelors/Family,TENANT PREFERRED_Family,POINT OF CONTACT_Contact Agent,POINT OF CONTACT_Contact Builder,POINT OF CONTACT_Contact Owner
1,3,1208,3,200000,False,False,False,False,False,True,...,True,False,False,True,True,False,False,True,False,False
2,3,1350,3,200000,False,True,False,False,False,False,...,True,True,False,False,True,False,False,True,False,False
3,3,1350,3,200000,False,True,False,False,False,False,...,True,True,False,False,True,False,False,True,False,False
6,4,1900,3,200000,False,False,False,False,False,True,...,True,True,False,False,False,True,False,True,False,False
17,3,1300,3,190000,False,True,False,False,False,False,...,True,True,False,False,True,False,False,True,False,False


In [20]:
X = df_encoded.iloc[:, :-1]
y = df_encoded.iloc[:, -1]

In [ ]:
X

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics
# Create a linear regression model
model = LinearRegression()

# Train the model using the training sets
model.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = model.predict(X_test)

# Calculate metrics
r2 = metrics.r2_score(y_test, y_pred)  # R-Squared
print('R-Squared:', r2)

adjusted_r2 = 1 - (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)  # Adjusted R-Squared
print('Adjusted R-Squared:', adjusted_r2)

mse = metrics.mean_squared_error(y_test, y_pred)  # Mean Squared Error
print('Mean Squared Error:', mse)

rmse = np.sqrt(mse)  # Root Mean Squared Error
print('Root Mean Squared Error:', rmse)

mae = metrics.mean_absolute_error(y_test, y_pred)  # Mean Absolute Error
print('Mean Absolute Error:', mae)

R-Squared: 1.0
Adjusted R-Squared: 1.0
Mean Squared Error: 2.2798165268648594e-26
Root Mean Squared Error: 1.509906131805835e-13
Mean Absolute Error: 1.1593514651086659e-13


In [ ]:
#Analyze the proportion or percentage of each category in the dataset.
category_proportions = data['BATHROOM'].value_counts(normalize=True)
print(category_proportions*100)

In [ ]:
from scipy import stats
z_scores = stats.zscore(df['BATHROOM'])
outliers = df[(z_scores < -3) | (z_scores > 3)]
outliers